In [3]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())
pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [4]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)
df = df.filter(pl.col("stran") > 30)

df = df.drop_nulls(subset=['100_7'])
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["100_a","245_a"], keep="first")
print(len(df))

716789
449395


In [5]:
wikid = pl.read_parquet(os.path.join('data', 'wikidata.parquet'))
wikid.sample(5)

024_a,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en,__index_level_0__
str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str,str
"""Q95152787""",null,null,null,null,[],"""Dana Dobřichovská""","""Dana Dobřichovská""",null,null,null,"""Narozena 19.7.1966 v Havlíčkov…",null,null,null,null,null,null,null,null,null,null,null,"""žena""","[""Havlíčkův Brod""]",null,1966.0,"""+1966-07-19T00:00:00Z""",null,null,null,null,null,"""js20060210002"""
"""Q4760601""",null,null,null,null,"[""fr"", ""arz"", … ""no""]","""Andy Diggle""","""Andy Diggle""",null,"[""Spojené království""]",null,null,"""British writer""",null,null,"[""storyboard artist"", ""komiksový scenárista"", ""editor""]",null,null,null,null,"""andydiggle""",null,null,"""muž""","[""Londýn""]",null,1954.0,"""+1954-01-09T00:00:00Z""",null,null,"""http://www.andydiggle.com""",null,"""https://en.wikipedia.org/wiki/…","""xx0118916"""
"""Q32859503""",null,null,null,null,"[""en"", ""de"", ""fr""]","""Gerd Vogel""","""Gerd-Helge Vogel""",null,"[""Německá demokratická republika""]",null,null,"""German art historian""",null,null,"[""historik umění""]",null,null,null,null,null,null,null,"""muž""","[""Cvikov""]",null,1951.0,"""+1951-04-18T00:00:00Z""",null,null,null,null,"""https://en.wikipedia.org/wiki/…","""jcu2013786239"""
"""Q124741351""",null,null,null,null,[],"""Nikola Prokešová""","""Nikola Prokešová""",null,null,null,"""MUDr., lékařka specializovaná …",null,null,null,"[""lékař""]",null,null,null,null,null,null,null,"""žena""",null,null,null,null,null,null,null,null,null,"""xx0269425"""
"""Q1987959""","[""Manly Wade Wellman Award""]",null,null,null,"[""bg"", ""nl"", … ""en""]","""A. J. (Andrew James) Hartley""","""A. J. Hartley""",null,"[""Spojené státy americké""]",null,"""americký spisovatel""","""American writer""",null,null,"[""romanopisec"", ""spisovatel"", … ""vysokoškolský učitel""]",null,"[""Ushaw College""]",null,null,null,null,null,"""muž""","[""Preston""]",null,1964.0,"""+1964-08-07T00:00:00Z""",null,null,"""https://ajhartley.net/""",null,"""https://en.wikipedia.org/wiki/…","""xx0100706"""


In [6]:
narozeni_wikid = wikid.explode('w_misto_narozeni').rename({'__index_level_0__':'100_7','w_misto_narozeni':'rodna_obec'}).select(pl.col(['100_7','rodna_obec'])).drop_nulls()
narozeni_wikid.sample(5)

100_7,rodna_obec
str,str
"""mzk2005271963""","""Praha"""
"""jk01082565""","""Nový Hradec Králové"""
"""jn19990210827""","""Petřkovice"""
"""xx0000025""","""Krefeld"""
"""jo2008475983""","""Praha"""


In [7]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

364420

In [8]:
df = df.filter(pl.col("100_7").is_in(cesi))

In [9]:
def jenom_obec(adresa):
    if "," in adresa:
        return adresa.split(",")[0]
    else:
        return ""

In [10]:
narozeni_aut = aut.explode(
    '100_7'
).explode(
    "370_a"
).drop_nulls(
    subset=['370_a']
).with_columns(
    pl.col("370_a").map_elements(
        jenom_obec
    ).alias("rodna_obec")
).select(
    pl.col(['100_7','rodna_obec'])
)

narozeni_aut

100_7,rodna_obec
str,str
"""jk01010004""","""Odrlice"""
"""jk01010022""","""Příbram"""
"""jk01010030""","""Praha"""
"""jk01010034""","""Štítina"""
"""jk01010104""","""Praha"""
"""jk01010058""","""Praha"""
"""jk01010070""","""Bludov"""
"""jk01010073""","""Plzeň"""
"""jk01010087""","""Hlinsko"""


In [11]:
def specialized_join(df1: pl.DataFrame, df2: pl.DataFrame) -> pl.DataFrame:
    # Perform outer join
    joined = df1.join(
        df2,
        on="100_7",
        how="outer",
        suffix="_df2"
    )
    
    # Coalesce all columns, including the ID column
    result = joined.with_columns([
        pl.coalesce("100_7", "100_7_df2").alias("100_7"),
        pl.coalesce("rodna_obec", "rodna_obec_df2").alias("rodna_obec")
    ])
    
    # Drop the duplicate columns
    result = result.drop(["100_7_df2", "rodna_obec_df2"])
    
    return result

In [12]:
rodne_obce = specialized_join(narozeni_aut, narozeni_wikid)

In [13]:
df = df.join(rodne_obce, on='100_7', how='left')

In [14]:
sjednoceni = {
    "Brandýs nad Labem": "Brandýs nad Labem-Stará Boleslav",
    "Stará Boleslav": "Brandýs nad Labem-Stará Boleslav",
    "Vinohrady": "Praha",
    "Frýdek": "Frýdek-Místek",
    "Místek": "Frýdek-Místek",
    "Smíchov": "Praha",
    "Žižkov": "Praha",
    "Moravská Ostrava": "Ostrava",
    "Slezská Ostrava": "Ostrava",
    "Malá Strana": "Praha",
    "Nové Město": "Praha",
    "Karlín": "Praha",
    "Vítkovice": "Ostrava",
    "Josefov": "Praha",
    "Vršovice": "Praha",
    "Tuřany": "Brno",
    "Bohunice": "Brno",
    "Židenice": "Brno",
    "Královo Pole": "Brno",
    "Husovice": "Brno",
    "Poruba": "Ostrava",
    "Černovice": "Brno",
    "Jundrov": "Brno",
    "Bystrc": "Brno",
    "Brno-Bosonohy": "Brno",
    "Brno-Židenice": "Brno",
    "Staré Brno": "Brno",
    "Praha-Zbraslav":"Praha",
    "Praha-Slivenec": "Praha",
    "Praha II": "Praha"
}

In [15]:
df = df.with_columns(pl.col('rodna_obec').replace(
    sjednoceni))

In [16]:
df.group_by('rodna_obec').len().sort(by='len',descending=True).head(10)

rodna_obec,len
str,u32
null,106898
"""Praha""",52846
"""Brno""",11598
"""Plzeň""",5015
"""Ostrava""",3774
"""Olomouc""",3157
"""České Budějovice""",2857
"""Hradec Králové""",2467
"""Pardubice""",1966


In [17]:
df.group_by('rodna_obec').len().filter(pl.col("rodna_obec").str.contains("Praha"))

rodna_obec,len
str,u32
"""Praha-Řeporyje""",1
"""Praha 2""",1
"""Praha""",52846
"""Praha XIII""",3
"""Praha XIX""",3


In [18]:
df.group_by('rodna_obec').len().filter(pl.col("rodna_obec").str.contains("Bystr"))

rodna_obec,len
str,u32
"""Bystré""",15
"""Banská Bystrica""",72
"""Bystrovany""",5
"""Bystročice""",1
"""Považská Bystrica""",37
"""Bystrá nad Jizerou""",14


## Velikost a umístění měst

In [20]:
import requests
from bs4 import BeautifulSoup

In [21]:
r = requests.get("https://cs.wikipedia.org/wiki/Seznam_m%C4%9Bst_v_%C4%8Cesku_podle_po%C4%8Dtu_obyvatel")
soup = BeautifulSoup(r.text, 'html.parser')
mesta = {}
tabulka = soup.find(class_="sortable")
for radek in tabulka.find_all('tr'):
    if 'Rozloha' not in radek.text:
        mesta[radek.find_all('td')[1].text.strip()] = int(''.join([x for x in radek.find_all('td')[3].text.strip() if x.isdigit()]))
print(len(mesta))
r = requests.get("https://cs.wikipedia.org/wiki/Seznam_nejlidnat%C4%9Bj%C5%A1%C3%ADch_obc%C3%AD_v_%C4%8Cesku_bez_statusu_m%C4%9Bsta")
soup = BeautifulSoup(r.text, 'html.parser')
vsi = {}
tabulka = soup.find(class_="sortable")
for radek in tabulka.find_all('tr'):
    try:
        if 'Početoby' not in radek.text:
            ves = radek.find_all('td')[0].text.strip()
            velikost = int(''.join([x for x in radek.find_all('td')[1].text.strip() if x.isdigit()]))
            vsi[ves] = velikost
    except:
        print(radek.text)
len(vsi)
obce = pd.concat([pd.Series(mesta), pd.Series(vsi)])
vsechny_obce = obce.index.to_list()
len(vsechny_obce)

607


707

In [22]:
lokality = pd.read_json(os.path.join('../ceske-obce/ceske-obce.json'))

In [23]:
lokality

,name,wikidata_id,coordinates,population,area,region,district
1097,Praha,Q1085,Point(14.421388888 50.0875),1384732,496.210000,Czech Republic,Unknown
1143,Brno,Q14960,Point(16.608333333 49.195277777),400566,230.182739,Brno-City District,Unknown
1136,Ostrava,Q8385,Point(18.2925 49.835555555),284765,214.229602,Ostrava-City District,Unknown
1086,Plzeň,Q43453,Point(13.3825 49.741388888),185599,137.671042,Plzeň-City District,Unknown
952,Liberec,Q146351,Point(15.05 50.766666666),107982,106.087154,Liberec District,Unknown
...,...,...,...,...,...,...,...
526,Hill fort Chlum,Q110290703,Point(14.926382303 50.378408905),0,0.000000,Nepřevázka,Unknown
273,Q37453718,Q37453718,Point(13.8794367 49.0689152),0,0.000000,Lštění,Unknown
527,Q47035482,Q47035482,Point(13.51383 49.7611342),0,0.000000,Dýšina,Unknown
528,Q106630428,Q106630428,Point(14.5871 50.0557),0,0.000000,Dubeč,Unknown


## Různé pohledy

In [25]:
do_10000 = pl.from_pandas(lokality).filter(pl.col('population').is_between(5,10000)).select(pl.col('name')).to_series().to_list()
do_1000 = pl.from_pandas(lokality).filter(pl.col('population').is_between(5,1000)).select(pl.col('name')).to_series().to_list()
nad_10000 = pl.from_pandas(lokality).filter(pl.col('population') > 10000).select(pl.col('name')).to_series().to_list()
nad_1000 = pl.from_pandas(lokality).filter(pl.col('population') > 1000).select(pl.col('name')).to_series().to_list()
nad_5000 = pl.from_pandas(lokality).filter(pl.col('population') > 5000).select(pl.col('name')).to_series().to_list()

### Knih na obec

In [27]:
df.group_by("rodna_obec").len().sort(by='len',descending=True).head(100)

rodna_obec,len
str,u32
null,106898
"""Praha""",52846
"""Brno""",11598
"""Plzeň""",5015
"""Ostrava""",3774
"""Olomouc""",3157
"""České Budějovice""",2857
"""Hradec Králové""",2467
"""Pardubice""",1966


### Rodáků na obec (neřeší se hody jmen)

In [29]:
df.group_by("rodna_obec").agg(pl.col('100_a').unique().len()).sort(by='100_a',descending=True).head(100)

rodna_obec,100_a
str,u32
null,41289
"""Praha""",9694
"""Brno""",2580
"""Plzeň""",1087
"""Ostrava""",941
"""Olomouc""",673
"""České Budějovice""",585
"""Hradec Králové""",553
"""Pardubice""",474


Videň a Bratislava víc než Karlovy Vary.

### Knih na č. obec do 10000 obyvatel (Vídeň je nesmysl)

In [32]:
df.filter(pl.col('rodna_obec').is_in(do_10000)).group_by('rodna_obec').len().sort(by='len',descending=True).head(10)

rodna_obec,len
str,u32
"""Vídeň""",1116
"""Hořice""",651
"""Nové Město na Moravě""",621
"""Polička""",584
"""Nivnice""",457
"""Ivančice""",436
"""Košice""",387
"""Chlumec nad Cidlinou""",375
"""Jilemnice""",343


### Rodáků na obec do 10k

In [34]:
df.filter(pl.col('rodna_obec').is_in(do_10000)).group_by('rodna_obec').agg(pl.col('100_a').unique().len()).sort(by='100_a',descending=True).head(10)

rodna_obec,100_a
str,u32
"""Vídeň""",212
"""Nové Město na Moravě""",176
"""Polička""",116
"""Jilemnice""",95
"""Hořice""",91
"""Ivančice""",89
"""Košice""",84
"""Dačice""",76
"""Chlumec nad Cidlinou""",64


In [35]:
df.filter(pl.col('rodna_obec').is_in(do_1000)).group_by('rodna_obec').len().sort(by='len',descending=True).head(10)

rodna_obec,len
str,u32
"""Vídeň""",1116
"""Košice""",387
"""Havlíčkova Borová""",271
"""Petrovice""",149
"""Miletín""",143
"""Milavče""",124
"""Biskoupky""",121
"""Sedlec""",121
"""Skalička""",114


In [36]:
df.filter(pl.col('rodna_obec').is_in(do_1000)).group_by('rodna_obec').agg(pl.col('100_7').unique().len()).sort(by='100_7',descending=True).head(10)

rodna_obec,100_7
str,u32
"""Vídeň""",212
"""Košice""",84
"""Žilina""",29
"""Malenovice""",17
"""Sedlec""",16
"""Komárov""",13
"""Nové Dvory""",13
"""Křivoklát""",12
"""Petrovice""",11


In [37]:
rodactvo = df.group_by('rodna_obec').agg(pl.col('100_7').unique().len()).sort(by='100_7',descending=True).join(
    pl.from_pandas(lokality).select(pl.col(['name','population'])), left_on='rodna_obec', right_on='name', how="full"
).with_columns(
    (pl.col('100_7') / pl.col('population')).alias('podil')
).sort(by='podil',descending=True)

In [38]:
rodactvo.sample(10)

rodna_obec,100_7,name,population,podil
str,u32,str,i64,f64
"""Scarborough""",1,null,null,null
"""Malobratřice""",2,null,null,null
null,null,"""Želivsko""",36,null
null,null,"""Krompach""",178,null
"""Hlízov""",1,null,null,null
"""Krašovice""",2,null,null,null
"""Topoľčianky""",2,null,null,null
"""Lutyně""",1,null,null,null
null,null,"""Ústrašice""",409,null


In [39]:
rodactvo.filter(pl.col('rodna_obec').is_in(do_10000))

rodna_obec,100_7,name,population,podil
str,u32,str,i64,f64
"""Vídeň""",212,"""Vídeň""",456,0.464912
"""Nové Zámky""",8,"""Nové Zámky""",30,0.266667
"""Košice""",84,"""Košice""",782,0.107417
"""Minice""",2,"""Minice""",28,0.071429
"""Útěchovice""",4,"""Útěchovice""",61,0.065574
"""Nový Dvůr""",4,"""Nový Dvůr""",67,0.059701
"""Čilá""",1,"""Čilá""",18,0.055556
"""Krakov""",7,"""Krakov""",139,0.05036
"""Louka""",3,"""Louka""",61,0.04918


In [40]:
rodactvo.filter(pl.col('rodna_obec').is_in(nad_10000))

rodna_obec,100_7,name,population,podil
str,u32,str,i64,f64
"""Litomyšl""",164,"""Litomyšl""",10493,0.015629
"""Čáslav""",143,"""Čáslav""",10512,0.013604
"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,0.013459
"""Jičín""",215,"""Jičín""",16230,0.013247
"""Kyjov""",141,"""Kyjov""",10799,0.013057
"""Boskovice""",143,"""Boskovice""",12190,0.011731
"""Vysoké Mýto""",142,"""Vysoké Mýto""",12412,0.011441
"""Turnov""",157,"""Turnov""",14502,0.010826
"""Uherské Hradiště""",260,"""Uherské Hradiště""",24933,0.010428


In [41]:
rodactvo.filter(pl.col('rodna_obec').is_in(nad_1000))

rodna_obec,100_7,name,population,podil
str,u32,str,i64,f64
"""Čeladná""",61,"""Čeladná""",2904,0.021006
"""Přílepy""",19,"""Přílepy""",1014,0.018738
"""Opočno""",57,"""Opočno""",3140,0.018153
"""Nové Město na Moravě""",176,"""Nové Město na Moravě""",9904,0.017771
"""Jilemnice""",96,"""Jilemnice""",5443,0.017637
"""Vysoké nad Jizerou""",23,"""Vysoké nad Jizerou""",1329,0.017306
"""Valtice""",59,"""Valtice""",3593,0.016421
"""Městec Králové""",46,"""Městec Králové""",2860,0.016084
"""Kněževes""",16,"""Kněževes""",1018,0.015717


In [42]:
rodactvo.filter(pl.col('rodna_obec').is_in(nad_5000))

rodna_obec,100_7,name,population,podil
str,u32,str,i64,f64
"""Nové Město na Moravě""",176,"""Nové Město na Moravě""",9904,0.017771
"""Jilemnice""",96,"""Jilemnice""",5443,0.017637
"""Litomyšl""",164,"""Litomyšl""",10493,0.015629
"""Čáslav""",143,"""Čáslav""",10512,0.013604
"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,0.013459
"""Jičín""",215,"""Jičín""",16230,0.013247
"""Polička""",117,"""Polička""",8939,0.013089
"""Kyjov""",141,"""Kyjov""",10799,0.013057
"""Boskovice""",143,"""Boskovice""",12190,0.011731


### Souřadnice

In [44]:
lokality

,name,wikidata_id,coordinates,population,area,region,district
1097,Praha,Q1085,Point(14.421388888 50.0875),1384732,496.210000,Czech Republic,Unknown
1143,Brno,Q14960,Point(16.608333333 49.195277777),400566,230.182739,Brno-City District,Unknown
1136,Ostrava,Q8385,Point(18.2925 49.835555555),284765,214.229602,Ostrava-City District,Unknown
1086,Plzeň,Q43453,Point(13.3825 49.741388888),185599,137.671042,Plzeň-City District,Unknown
952,Liberec,Q146351,Point(15.05 50.766666666),107982,106.087154,Liberec District,Unknown
...,...,...,...,...,...,...,...
526,Hill fort Chlum,Q110290703,Point(14.926382303 50.378408905),0,0.000000,Nepřevázka,Unknown
273,Q37453718,Q37453718,Point(13.8794367 49.0689152),0,0.000000,Lštění,Unknown
527,Q47035482,Q47035482,Point(13.51383 49.7611342),0,0.000000,Dýšina,Unknown
528,Q106630428,Q106630428,Point(14.5871 50.0557),0,0.000000,Dubeč,Unknown


In [45]:
def souradnice(point):
    point = str(point)
    return {'longitude': float(point.split('(')[1].split(' ')[0]), 'latitude': float(point.split(' ')[1].split(')')[0])}

In [46]:
souradnice("Point(14.5871 50.0557)")

{'longitude': 14.5871, 'latitude': 50.0557}

In [47]:
zkraceni = {
    "Dvůr Králové nad Labem":"Dvůr Králové",
    "Nové Město na Moravě": "Nové Město n. M."
}

In [48]:
nesmysly = ["Vídeň", "Košice", "Nové Zámky", "Krakov", "Žilina"]

In [49]:
df.group_by("rodna_obec").len().rename({'len':'knih'}).join(
    df.group_by('rodna_obec').agg(pl.col('100_a').unique().len()).rename({'100_a':'rodaku'}), on='rodna_obec', how='full').join(
    pl.from_pandas(lokality).select(pl.col(['name','population','coordinates'])), left_on='rodna_obec', right_on='name', how='full'
).rename({'population':'obyvatel'}
)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,coordinates
str,u32,str,u32,str,i64,str
"""Kunžak""",11,"""Kunžak""",6,"""Kunžak""",1430,"""Point(15.190299026 49.12124885…"
"""Měšice""",3,"""Měšice""",2,"""Měšice""",2195,"""Point(14.519992873 50.19806789…"
"""Lubina""",7,"""Lubina""",2,null,null,null
"""Jílové u Prahy""",68,"""Jílové u Prahy""",7,"""Jílové u Prahy""",5227,"""Point(14.493387704 49.89552087…"
"""Dobroměřice""",2,"""Dobroměřice""",1,null,null,null
"""Paseky""",1,"""Paseky""",1,"""Paseky""",194,"""Point(14.254842088 49.25171718…"
"""Dvorec""",26,"""Dvorec""",4,null,null,null
"""Dolní Studénky""",13,"""Dolní Studénky""",6,"""Dolní Studénky""",1427,"""Point(16.971128477 49.93507922…"
"""Pruské""",1,"""Pruské""",1,null,null,null


In [50]:
final = df.group_by("rodna_obec").len().rename({'len':'knih'}).join(
    df.group_by('rodna_obec').agg(pl.col('100_a').unique().len()).rename({'100_a':'rodaku'}), on='rodna_obec', how='full').join(
    pl.from_pandas(lokality).select(pl.col(['name','population','coordinates'])), left_on='rodna_obec', right_on='name', how='full'
).rename({'population':'obyvatel'}
).with_columns(
    (pl.col('rodaku') / pl.col('obyvatel')).alias('rodaku/obyvatel')
).with_columns((pl.col('knih') / pl.col('obyvatel')).alias('knih/obyvatel'
)).with_columns(pl.col("rodna_obec").map_elements(len).alias("delka_nazvu")
).with_columns(pl.col('coordinates').map_elements(souradnice)).unnest('coordinates').with_columns(pl.col('rodna_obec').replace(
    zkraceni)).filter(pl.col("obyvatel") > 0).filter(
    ~pl.col("rodna_obec").is_in(nesmysly)
    )
final.sort(by='knih',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Praha""",52846,"""Praha""",9694,"""Praha""",1384732,14.421389,50.0875,0.007001,0.038163,5
"""Brno""",11598,"""Brno""",2580,"""Brno""",400566,16.608333,49.195278,0.006441,0.028954,4
"""Plzeň""",5015,"""Plzeň""",1087,"""Plzeň""",185599,13.3825,49.741389,0.005857,0.027021,5
"""Ostrava""",3774,"""Ostrava""",941,"""Ostrava""",284765,18.2925,49.835556,0.003304,0.013253,7
"""Olomouc""",3157,"""Olomouc""",673,"""Olomouc""",102293,17.250833,49.593889,0.006579,0.030862,7
"""České Budějovice""",2857,"""České Budějovice""",585,"""České Budějovice""",97377,14.474722,48.974722,0.006008,0.02934,16
"""Hradec Králové""",2467,"""Hradec Králové""",553,"""Hradec Králové""",93906,15.831944,50.209167,0.005889,0.026271,14
"""Pardubice""",1966,"""Pardubice""",474,"""Pardubice""",92362,15.77916,50.03861,0.005132,0.021286,9
"""Písek""",1844,"""Písek""",284,"""Písek""",30986,14.147531,49.308881,0.009165,0.059511,5


In [51]:
final.drop_nulls().sort(by='delka_nazvu',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Nová Ves u Nového Města na Mor…",2,"""Nová Ves u Nového Města na Mor…",1,"""Nová Ves u Nového Města na Mor…",670,16.084136,49.539967,0.001493,0.002985,33
"""Brandýs nad Labem-Stará Bolesl…",547,"""Brandýs nad Labem-Stará Bolesl…",114,"""Brandýs nad Labem-Stará Bolesl…",20073,14.663333,50.187161,0.005679,0.027251,32
"""Blatnice pod Svatým Antonínkem""",10,"""Blatnice pod Svatým Antonínkem""",3,"""Blatnice pod Svatým Antonínkem""",1926,17.465959,48.946732,0.001558,0.005192,30
"""Jakubov u Moravských Budějovic""",3,"""Jakubov u Moravských Budějovic""",1,"""Jakubov u Moravských Budějovic""",677,15.761429,49.08106,0.001477,0.004431,30
"""Dobrá Voda u Českých Budějovic""",2,"""Dobrá Voda u Českých Budějovic""",2,"""Dobrá Voda u Českých Budějovic""",2669,14.524943,48.968284,0.000749,0.000749,30
"""Rokytnice v Orlických horách""",3,"""Rokytnice v Orlických horách""",2,"""Rokytnice v Orlických horách""",2164,16.465745,50.164761,0.000924,0.001386,28
"""Klášter Hradiště nad Jizerou""",3,"""Klášter Hradiště nad Jizerou""",2,"""Klášter Hradiště nad Jizerou""",1018,14.944816,50.52347,0.001965,0.002947,28
"""Kostelec nad Černými lesy""",142,"""Kostelec nad Černými lesy""",18,"""Kostelec nad Černými lesy""",4194,14.859283,49.994101,0.004292,0.033858,25
"""Útěchovice pod Stražištěm""",14,"""Útěchovice pod Stražištěm""",2,"""Útěchovice pod Stražištěm""",115,15.029134,49.538568,0.017391,0.121739,25


In [52]:
final.filter(pl.col('knih').is_null())

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64


In [53]:
final.filter(pl.col('obyvatel') > 0).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Louka""",51,"""Louka""",3,"""Louka""",61,16.431079,49.5292,0.04918,0.836066,5
"""Benátky""",86,"""Benátky""",5,"""Benátky""",122,15.736071,50.310237,0.040984,0.704918,7
"""Biskoupky""",121,"""Biskoupky""",2,"""Biskoupky""",188,16.281293,49.097724,0.010638,0.643617,9
"""Třebonín""",80,"""Třebonín""",3,"""Třebonín""",138,15.312978,49.869759,0.021739,0.57971,8
"""Karlov""",58,"""Karlov""",4,"""Karlov""",117,15.914477,49.647047,0.034188,0.495726,6
"""Útěchovice""",28,"""Útěchovice""",4,"""Útěchovice""",61,15.116417,49.478133,0.065574,0.459016,10
"""Dědice""",59,"""Dědice""",5,"""Dědice""",135,15.727762,49.029751,0.037037,0.437037,6
"""Třebíz""",103,"""Třebíz""",3,"""Třebíz""",240,13.990873,50.269701,0.0125,0.429167,6
"""Minice""",12,"""Minice""",2,"""Minice""",28,14.012172,49.489925,0.071429,0.428571,6


In [54]:
final.filter(pl.col('obyvatel') > 0).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Minice""",12,"""Minice""",2,"""Minice""",28,14.012172,49.489925,0.071429,0.428571,6
"""Útěchovice""",28,"""Útěchovice""",4,"""Útěchovice""",61,15.116417,49.478133,0.065574,0.459016,10
"""Nový Dvůr""",24,"""Nový Dvůr""",4,"""Nový Dvůr""",67,15.110943,50.232204,0.059701,0.358209,9
"""Čilá""",5,"""Čilá""",1,"""Čilá""",18,13.742122,49.961752,0.055556,0.277778,4
"""Louka""",51,"""Louka""",3,"""Louka""",61,16.431079,49.5292,0.04918,0.836066,5
"""Komárov""",72,"""Komárov""",13,"""Komárov""",309,17.568761,49.150692,0.042071,0.23301,7
"""Benátky""",86,"""Benátky""",5,"""Benátky""",122,15.736071,50.310237,0.040984,0.704918,7
"""Horní Vilémovice""",30,"""Horní Vilémovice""",3,"""Horní Vilémovice""",78,15.880829,49.291604,0.038462,0.384615,16
"""Budyně""",15,"""Budyně""",2,"""Budyně""",53,14.07118,49.146383,0.037736,0.283019,6


In [55]:
final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Kněževes""",164,"""Kněževes""",16,"""Kněževes""",1018,13.637178,50.146677,0.015717,0.1611,8
"""Malé Svatoňovice""",228,"""Malé Svatoňovice""",7,"""Malé Svatoňovice""",1591,16.049874,50.533982,0.0044,0.143306,16
"""Nivnice""",457,"""Nivnice""",6,"""Nivnice""",3373,17.647573,48.974649,0.001779,0.135488,7
"""Klenčí pod Čerchovem""",185,"""Klenčí pod Čerchovem""",10,"""Klenčí pod Čerchovem""",1459,12.814762,49.434952,0.006854,0.126799,20
"""Přílepy""",116,"""Přílepy""",19,"""Přílepy""",1014,17.615931,49.319983,0.018738,0.114398,7
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Nové Hrady""",264,"""Nové Hrady""",11,"""Nové Hrady""",2506,14.778383,48.789702,0.004389,0.105347,10
"""Kvasiny""",167,"""Kvasiny""",6,"""Kvasiny""",1597,16.263302,50.212529,0.003757,0.104571,7
"""Trhová Kamenice""",106,"""Trhová Kamenice""",7,"""Trhová Kamenice""",1020,15.815995,49.786838,0.006863,0.103922,15


In [56]:
final.filter(pl.col('obyvatel') > 10000).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Jičín""",1114,"""Jičín""",215,"""Jičín""",16230,15.351653,50.437045,0.013247,0.068638,5
"""Boskovice""",777,"""Boskovice""",143,"""Boskovice""",12190,16.660002,49.487537,0.011731,0.063741,9
"""Písek""",1844,"""Písek""",284,"""Písek""",30986,14.147531,49.308881,0.009165,0.059511,5
"""Vysoké Mýto""",737,"""Vysoké Mýto""",141,"""Vysoké Mýto""",12412,16.161752,49.95324,0.01136,0.059378,11
"""Turnov""",843,"""Turnov""",156,"""Turnov""",14502,15.156899,50.587342,0.010757,0.05813,6
"""Kutná Hora""",1241,"""Kutná Hora""",215,"""Kutná Hora""",21556,15.268226,49.948445,0.009974,0.057571,10
"""Rychnov nad Kněžnou""",651,"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,16.274955,50.162871,0.013459,0.056896,19


In [57]:
final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Čeladná""",179,"""Čeladná""",61,"""Čeladná""",2904,18.337616,49.548715,0.021006,0.061639,7
"""Přílepy""",116,"""Přílepy""",19,"""Přílepy""",1014,17.615931,49.319983,0.018738,0.114398,7
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Nové Město n. M.""",621,"""Nové Město na Moravě""",176,"""Nové Město na Moravě""",9904,16.074221,49.561482,0.017771,0.062702,20
"""Jilemnice""",343,"""Jilemnice""",95,"""Jilemnice""",5443,15.506628,50.608945,0.017454,0.063017,9
"""Vysoké nad Jizerou""",124,"""Vysoké nad Jizerou""",23,"""Vysoké nad Jizerou""",1329,15.401628,50.685639,0.017306,0.093303,18
"""Valtice""",191,"""Valtice""",59,"""Valtice""",3593,16.754981,48.740717,0.016421,0.053159,7
"""Městec Králové""",169,"""Městec Králové""",46,"""Městec Králové""",2860,15.297657,50.207236,0.016084,0.059091,14
"""Kněževes""",164,"""Kněževes""",16,"""Kněževes""",1018,13.637178,50.146677,0.015717,0.1611,8


In [58]:
final.filter(pl.col('obyvatel') > 10000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Rychnov nad Kněžnou""",651,"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,16.274955,50.162871,0.013459,0.056896,19
"""Jičín""",1114,"""Jičín""",215,"""Jičín""",16230,15.351653,50.437045,0.013247,0.068638,5
"""Kyjov""",590,"""Kyjov""",141,"""Kyjov""",10799,17.122532,49.010193,0.013057,0.054635,5
"""Boskovice""",777,"""Boskovice""",143,"""Boskovice""",12190,16.660002,49.487537,0.011731,0.063741,9
"""Vysoké Mýto""",737,"""Vysoké Mýto""",141,"""Vysoké Mýto""",12412,16.161752,49.95324,0.01136,0.059378,11
"""Turnov""",843,"""Turnov""",156,"""Turnov""",14502,15.156899,50.587342,0.010757,0.05813,6
"""Uherské Hradiště""",803,"""Uherské Hradiště""",260,"""Uherské Hradiště""",24933,17.459691,49.069758,0.010428,0.032206,16


In [59]:
final.filter(pl.col('obyvatel') > 50000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Praha""",52846,"""Praha""",9694,"""Praha""",1384732,14.421389,50.0875,0.007001,0.038163,5
"""Olomouc""",3157,"""Olomouc""",673,"""Olomouc""",102293,17.250833,49.593889,0.006579,0.030862,7
"""Brno""",11598,"""Brno""",2580,"""Brno""",400566,16.608333,49.195278,0.006441,0.028954,4
"""Zlín""",1626,"""Zlín""",463,"""Zlín""",74255,17.666944,49.233056,0.006235,0.021898,4
"""České Budějovice""",2857,"""České Budějovice""",585,"""České Budějovice""",97377,14.474722,48.974722,0.006008,0.02934,16
"""Hradec Králové""",2467,"""Hradec Králové""",553,"""Hradec Králové""",93906,15.831944,50.209167,0.005889,0.026271,14
"""Plzeň""",5015,"""Plzeň""",1087,"""Plzeň""",185599,13.3825,49.741389,0.005857,0.027021,5
"""Opava""",1069,"""Opava""",295,"""Opava""",55600,17.904444,49.938056,0.005306,0.019227,5
"""Pardubice""",1966,"""Pardubice""",474,"""Pardubice""",92362,15.77916,50.03861,0.005132,0.021286,9


In [60]:
final.filter(pl.col("rodna_obec") == "Vídeň")

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64


In [61]:
final.filter(pl.col("rodna_obec") == "Nové Město nad Metují")

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Nové Město nad Metují""",207,"""Nové Město nad Metují""",33,"""Nové Město nad Metují""",9304,16.151547,50.344599,0.003547,0.022248,21


In [62]:
final.filter(pl.col("rodna_obec") == "Nové Město")

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64


In [63]:
pl.from_pandas(lokality).filter(pl.col("name").str.contains("Stará Paka"))

name,wikidata_id,coordinates,population,area,region,district
str,str,str,i64,f64,str,str


In [64]:
final.filter(pl.col("rodna_obec").str.contains("Jaroměřice")).sort(by="rodaku",descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Jaroměřice nad Rokytnou""",48,"""Jaroměřice nad Rokytnou""",15,"""Jaroměřice nad Rokytnou""",4085,15.893318,49.094127,0.003672,0.01175,23


In [65]:
df.filter(pl.col("rodna_obec") == 'Kroměříž').filter(pl.col("100_a").str.contains("Spá")).group_by(["rodna_obec","100_a"]).len().rename({'len':'knih'}).sort(by="knih",descending=True)

rodna_obec,100_a,knih
str,str,u32
"""Kroměříž""","""Spáčil, Bedřich,""",7


In [66]:
top_bez = rodactvo.filter(pl.col("100_7").is_null()).sort(by="population",descending=True).join(
    pl.from_pandas(lokality).select(pl.col(['name','population','coordinates'])), left_on='name', right_on='name', how='left'
).with_columns(pl.col('coordinates').map_elements(souradnice)).unnest('coordinates').with_columns(pl.col('rodna_obec').replace(
    zkraceni)).filter(~pl.col('name').is_in(["Brno-Žebětín","Baška"])).head(15)

top_bez

rodna_obec,100_7,name,population,podil,population_right,longitude,latitude
str,u32,str,i64,f64,i64,f64,f64
null,null,"""Štětí""",8609,null,8609,14.374305,50.45306
null,null,"""Modřice""",5656,null,5656,16.614424,49.127917
null,null,"""Horní Slavkov""",5438,null,5438,12.807658,50.138745
null,null,"""Bor""",5132,null,5132,12.775216,49.71171
null,null,"""Meziboří""",4587,null,4587,13.598802,50.621246
null,null,"""Bolatice""",4501,null,4501,18.083636,49.951712
null,null,"""Kynšperk nad Ohří""",4496,null,4496,12.530346,50.119046
null,null,"""Chlumec""",4280,null,4280,13.939746,50.699804
null,null,"""Nová Role""",4244,null,4244,12.784302,50.27103


## Mapy

In [68]:
import requests
from bs4 import BeautifulSoup

In [69]:
import re

In [70]:
r = requests.get('https://www.modrykonik.cz/rodim/seznam-porodnic/')
soup = BeautifulSoup(r.text)

In [71]:
odkazy = soup.find_all('a')
porodnice = [o.text.replace('Porodnice','') for o in odkazy if 'Porodnice' in o.text[0:9]]
porodnice = [re.sub(r'[0123456789]','', p).split('-')[0].strip() for p in porodnice]
porodnice = list(set([p for p in porodnice if len(p) > 0]))
porodnice

['Vrchlabí',
 'Zlín',
 'Vsetín',
 'Brno',
 'Valašské Meziříčí',
 'Ostrava Vítkovice',
 'Jilemnice',
 'Opočno',
 'Benešov',
 'Klatovy',
 'Strakonice',
 'Ostrava',
 'Olomouc',
 'Karlovy Vary',
 'Neratovice',
 'Prachatice',
 'Roudnice nad Labem',
 'Kyjov',
 'Mladá Boleslav',
 'Litoměřice',
 'Chrudim',
 'Třebíč',
 'Jeseník',
 'Jablonec nad Nisou',
 'Nový Jičín',
 'Nymburk',
 'Kadaň',
 'Ostrov',
 'Šternberk',
 'Kolín',
 'Turnov',
 'Rokycany',
 'Nové Město na Moravě',
 'Pardubice',
 'Most',
 'Jihlava',
 'Jindřichův Hradec',
 'Vyškov',
 'Uherské Hradiště',
 'Frýdek Místek',
 'Znojmo',
 'Rychnov nad Kněžnou',
 'Rakovník',
 'Žatec',
 'Ústí nad Labem',
 'Chomutov',
 'Kladno',
 'Hořovice',
 'Kroměříž',
 'Prostějov',
 'Svitavy',
 'Karviná',
 'Hradec Králové nebo Rychnov nad Kněžnou?',
 'Česky Krumlov',
 'Havlíčkův Brod',
 'Boskovice',
 'Havířov',
 'Pardubice PARTUS',
 'Krnov',
 'Domažlice',
 'Opava',
 'Litomyšl',
 'Teplice',
 'Rychnov nad Kněžnou, porodnice Pardubice',
 'Slaný',
 'Přerov',
 'Plzeň

In [72]:
porodnice = [p.split(' x ')[0] for p in porodnice] + [p.split(' x ')[1] for p in porodnice if 'x' in p]
porodnice = [p.split(',')[0] for p in porodnice]
porodnice = porodnice + [zkraceni[x] for x in porodnice if x in zkraceni.keys()]
porodnice = list(set(porodnice))
porodnice.sort()
porodnice

['Apolinář',
 'Benešov',
 'Boskovice',
 'Brandýs nad Labem',
 'Brno',
 'Břeclav',
 'Cheb',
 'Chomutov',
 'Chrudim',
 'Domažlice',
 'Děčín',
 'Frýdek Místek',
 'Havlíčkův Brod',
 'Havířov',
 'Hořovice',
 'Hradec Králové nebo Rychnov nad Kněžnou?',
 'Ivančice',
 'Jablonec nad Nisou',
 'Jeseník',
 'Jihlava',
 'Jilemnice',
 'Jindřichův Hradec',
 'Jičín',
 'Kadaň',
 'Karlovy Vary',
 'Karviná',
 'Kladno',
 'Klatovy',
 'Kolín',
 'Krnov',
 'Kroměříž',
 'Kyjov',
 'Liberec',
 'Litomyšl',
 'Litoměřice',
 'Mladá Boleslav',
 'Most',
 'Mělník',
 'Neratovice',
 'Nové Město n. M.',
 'Nové Město na Moravě',
 'Nový Jičín',
 'Nymburk',
 'Náchod',
 'Olomouc',
 'Opava',
 'Opočno',
 'Ostrava',
 'Ostrava Vítkovice',
 'Ostrov',
 'Pardubice',
 'Pardubice PARTUS',
 'Pelhřimov',
 'Plzeň',
 'Prachatice',
 'Praha',
 'Prostějov',
 'Písek',
 'Přerov',
 'Příbram',
 'Rakovník',
 'Rokycany',
 'Roudnice nad Labem',
 'Rumburk',
 'Rychnov nad Kněžnou',
 'Slaný',
 'Sokolov',
 'Stod',
 'Strakonice',
 'Svitavy',
 'Teplice',


In [73]:
graf1 = final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10)
graf1

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Kněževes""",164,"""Kněževes""",16,"""Kněževes""",1018,13.637178,50.146677,0.015717,0.1611,8
"""Malé Svatoňovice""",228,"""Malé Svatoňovice""",7,"""Malé Svatoňovice""",1591,16.049874,50.533982,0.0044,0.143306,16
"""Nivnice""",457,"""Nivnice""",6,"""Nivnice""",3373,17.647573,48.974649,0.001779,0.135488,7
"""Klenčí pod Čerchovem""",185,"""Klenčí pod Čerchovem""",10,"""Klenčí pod Čerchovem""",1459,12.814762,49.434952,0.006854,0.126799,20
"""Přílepy""",116,"""Přílepy""",19,"""Přílepy""",1014,17.615931,49.319983,0.018738,0.114398,7
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Nové Hrady""",264,"""Nové Hrady""",11,"""Nové Hrady""",2506,14.778383,48.789702,0.004389,0.105347,10
"""Kvasiny""",167,"""Kvasiny""",6,"""Kvasiny""",1597,16.263302,50.212529,0.003757,0.104571,7
"""Trhová Kamenice""",106,"""Trhová Kamenice""",7,"""Trhová Kamenice""",1020,15.815995,49.786838,0.006863,0.103922,15


In [74]:
graf2 = final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='rodaku/obyvatel',descending=True).head(10)
graf2

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Čeladná""",179,"""Čeladná""",61,"""Čeladná""",2904,18.337616,49.548715,0.021006,0.061639,7
"""Přílepy""",116,"""Přílepy""",19,"""Přílepy""",1014,17.615931,49.319983,0.018738,0.114398,7
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Nové Město n. M.""",621,"""Nové Město na Moravě""",176,"""Nové Město na Moravě""",9904,16.074221,49.561482,0.017771,0.062702,20
"""Jilemnice""",343,"""Jilemnice""",95,"""Jilemnice""",5443,15.506628,50.608945,0.017454,0.063017,9
"""Vysoké nad Jizerou""",124,"""Vysoké nad Jizerou""",23,"""Vysoké nad Jizerou""",1329,15.401628,50.685639,0.017306,0.093303,18
"""Valtice""",191,"""Valtice""",59,"""Valtice""",3593,16.754981,48.740717,0.016421,0.053159,7
"""Městec Králové""",169,"""Městec Králové""",46,"""Městec Králové""",2860,15.297657,50.207236,0.016084,0.059091,14
"""Kněževes""",164,"""Kněževes""",16,"""Kněževes""",1018,13.637178,50.146677,0.015717,0.1611,8


In [75]:
graf3 = final.filter(pl.col('obyvatel') > 10000).drop_nulls().sort(by='rodaku/obyvatel',descending=True).head(10)
graf3

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Rychnov nad Kněžnou""",651,"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,16.274955,50.162871,0.013459,0.056896,19
"""Jičín""",1114,"""Jičín""",215,"""Jičín""",16230,15.351653,50.437045,0.013247,0.068638,5
"""Kyjov""",590,"""Kyjov""",141,"""Kyjov""",10799,17.122532,49.010193,0.013057,0.054635,5
"""Boskovice""",777,"""Boskovice""",143,"""Boskovice""",12190,16.660002,49.487537,0.011731,0.063741,9
"""Vysoké Mýto""",737,"""Vysoké Mýto""",141,"""Vysoké Mýto""",12412,16.161752,49.95324,0.01136,0.059378,11
"""Turnov""",843,"""Turnov""",156,"""Turnov""",14502,15.156899,50.587342,0.010757,0.05813,6
"""Uherské Hradiště""",803,"""Uherské Hradiště""",260,"""Uherské Hradiště""",24933,17.459691,49.069758,0.010428,0.032206,16


In [76]:
graf4 = final.filter(pl.col('obyvatel') > 10000).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10)
graf4

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Jičín""",1114,"""Jičín""",215,"""Jičín""",16230,15.351653,50.437045,0.013247,0.068638,5
"""Boskovice""",777,"""Boskovice""",143,"""Boskovice""",12190,16.660002,49.487537,0.011731,0.063741,9
"""Písek""",1844,"""Písek""",284,"""Písek""",30986,14.147531,49.308881,0.009165,0.059511,5
"""Vysoké Mýto""",737,"""Vysoké Mýto""",141,"""Vysoké Mýto""",12412,16.161752,49.95324,0.01136,0.059378,11
"""Turnov""",843,"""Turnov""",156,"""Turnov""",14502,15.156899,50.587342,0.010757,0.05813,6
"""Kutná Hora""",1241,"""Kutná Hora""",215,"""Kutná Hora""",21556,15.268226,49.948445,0.009974,0.057571,10
"""Rychnov nad Kněžnou""",651,"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,16.274955,50.162871,0.013459,0.056896,19


In [77]:
import geopandas as gpd
import json

In [78]:
czech_geojson = gpd.read_file(os.path.join('mapy', 'czech_republic_simplified.json'), driver='GeoJSON')
czech_geojson = czech_geojson.to_crs('EPSG:4326')
czech_geojson = czech_geojson.buffer(0) ## toto bylo nutné, aby se neobarvovalo okolí ČR místo vnitřku 
czech_geojson_json = czech_geojson.to_json()

### Celkový pohled

In [80]:
nejvetsi = final.drop_nulls().sort(by="obyvatel", descending=True).head(500).select(pl.col("rodna_obec")).to_series().to_list()
nejvetsi

['Praha',
 'Brno',
 'Ostrava',
 'Plzeň',
 'Liberec',
 'Olomouc',
 'České Budějovice',
 'Hradec Králové',
 'Pardubice',
 'Ústí nad Labem',
 'Zlín',
 'Havířov',
 'Kladno',
 'Most',
 'Opava',
 'Jihlava',
 'Frýdek-Místek',
 'Teplice',
 'Karviná',
 'Karlovy Vary',
 'Chomutov',
 'Děčín',
 'Mladá Boleslav',
 'Jablonec nad Nisou',
 'Prostějov',
 'Přerov',
 'Česká Lípa',
 'Třebíč',
 'Tábor',
 'Třinec',
 'Znojmo',
 'Kolín',
 'Příbram',
 'Cheb',
 'Písek',
 'Trutnov',
 'Kroměříž',
 'Orlová',
 'Líšeň',
 'Vsetín',
 'Šumperk',
 'Uherské Hradiště',
 'Břeclav',
 'Havlíčkův Brod',
 'Hodonín',
 'Chrudim',
 'Český Těšín',
 'Nový Jičín',
 'Litoměřice',
 'Klatovy',
 'Valašské Meziříčí',
 'Krnov',
 'Strakonice',
 'Litvínov',
 'Sokolov',
 'Kopřivnice',
 'Kutná Hora',
 'Beroun',
 'Jindřichův Hradec',
 'Žďár nad Sázavou',
 'Bohumín',
 'Vyškov',
 'Mělník',
 'Blansko',
 'Brandýs nad Labem-Stará Boleslav',
 'Náchod',
 'Jirkov',
 'Žatec',
 'Kralupy nad Vltavou',
 'Kadaň',
 'Louny',
 'Hranice',
 'Otrokovice',
 'Bene

In [81]:
final

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Markvartice""",10,"""Markvartice""",3,"""Markvartice""",741,14.355381,50.775604,0.004049,0.013495,11
"""Michalovice""",1,"""Michalovice""",1,"""Michalovice""",221,15.524361,49.576986,0.004525,0.004525,11
"""Krahulčí""",18,"""Krahulčí""",3,"""Krahulčí""",633,15.414839,49.18183,0.004739,0.028436,8
"""Bezuchov""",1,"""Bezuchov""",1,"""Bezuchov""",187,17.6088,49.462731,0.005348,0.005348,8
"""Hradec""",19,"""Hradec""",2,"""Hradec""",613,13.121421,49.635158,0.003263,0.030995,6
"""Domašov nad Bystřicí""",2,"""Domašov nad Bystřicí""",1,"""Domašov nad Bystřicí""",491,17.44534,49.742305,0.002037,0.004073,20
"""Bošice""",3,"""Bošice""",2,"""Bošice""",377,13.840345,49.089374,0.005305,0.007958,6
"""Tištín""",1,"""Tištín""",1,"""Tištín""",465,17.165417,49.306902,0.002151,0.002151,6
"""Sehradice""",5,"""Sehradice""",1,"""Sehradice""",684,17.824167,49.144267,0.001462,0.00731,9


In [82]:
final.drop_nulls().filter(pl.col('rodna_obec').is_in(porodnice)).filter(pl.col('obyvatel') > 3000)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Ústí nad Orlicí""",483,"""Ústí nad Orlicí""",122,"""Ústí nad Orlicí""",14098,16.39367,49.973904,0.008654,0.03426,15
"""Kroměříž""",910,"""Kroměříž""",206,"""Kroměříž""",28089,17.393135,49.29786,0.007334,0.032397,8
"""Cheb""",231,"""Cheb""",70,"""Cheb""",32825,12.373996,50.079759,0.002133,0.007037,4
"""Ivančice""",436,"""Ivančice""",89,"""Ivančice""",9971,16.377529,49.101478,0.008926,0.043727,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Rokycany""",620,"""Rokycany""",120,"""Rokycany""",14386,13.594639,49.742794,0.008341,0.043097,8
"""Strakonice""",580,"""Strakonice""",146,"""Strakonice""",22522,13.902397,49.261498,0.006483,0.025753,10
"""Havlíčkův Brod""",1000,"""Havlíčkův Brod""",196,"""Havlíčkův Brod""",23746,15.580728,49.6079,0.008254,0.042112,14
"""Vsetín""",673,"""Vsetín""",174,"""Vsetín""",25255,17.996209,49.338703,0.00689,0.026648,6


In [83]:
lim = 3000
pocet_obci = len(final.filter(pl.col("obyvatel") >= lim))

vsemapka = alt.Chart(
    alt.Data(values=json.loads(czech_geojson_json)['features']),
    width=300,
    height=200,
    title=alt.Title(["Další mapa, na které jsou vidět Sudety"], 
                    subtitle=[f"Poměr spisovatelů a spisovatelek (publikujících česky","v letech 1800 až 2024) na počet obyvatel (k roku 2024).","Četnost je nejvyšší v tzv. Vysočinsko-orlickém pásu štěstí,","nejnižší je v Sudetech, do roku 1945 převážně německých.",f"(Mapa zobrazuje {pocet_obci} obcí s více než {lim} obyvateli.)"])
).mark_geoshape(
    filled=True,
    fill='#FFFFE4', #''#FFFFE4',  # Choose your desired fill color
    stroke='#EED801', #'#DB842F',
    strokeWidth=0.6
)

vseobce1 = alt.Chart(
    final.filter(pl.col('obyvatel') >= lim).to_pandas()
).mark_circle(
    color='#D6534B',  # red color
    opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('rodaku/obyvatel:Q', scale=alt.Scale(range=[0.05, 150]), legend=None),
    opacity=alt.Opacity('rodaku/obyvatel:Q', scale=alt.Scale(range=[0.6, 1]), legend=None),
)

hustota_do_grafu = (vsemapka + vseobce1)

hustota_do_grafu

alt.LayerChart(...)

In [84]:
final.filter(pl.col("rodna_obec").is_in(nejvetsi)).sort(by="rodaku/obyvatel",descending=True).head(100)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Čeladná""",179,"""Čeladná""",61,"""Čeladná""",2904,18.337616,49.548715,0.021006,0.061639,7
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Nové Město n. M.""",621,"""Nové Město na Moravě""",176,"""Nové Město na Moravě""",9904,16.074221,49.561482,0.017771,0.062702,20
"""Jilemnice""",343,"""Jilemnice""",95,"""Jilemnice""",5443,15.506628,50.608945,0.017454,0.063017,9
"""Valtice""",191,"""Valtice""",59,"""Valtice""",3593,16.754981,48.740717,0.016421,0.053159,7
"""Městec Králové""",169,"""Městec Králové""",46,"""Městec Králové""",2860,15.297657,50.207236,0.016084,0.059091,14
"""Počátky""",204,"""Počátky""",39,"""Počátky""",2500,15.240245,49.260258,0.0156,0.0816,7
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6


In [85]:
final.filter(pl.col("obyvatel") < 100000).sort(by="rodaku/obyvatel",descending=True).head(100)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Minice""",12,"""Minice""",2,"""Minice""",28,14.012172,49.489925,0.071429,0.428571,6
"""Útěchovice""",28,"""Útěchovice""",4,"""Útěchovice""",61,15.116417,49.478133,0.065574,0.459016,10
"""Nový Dvůr""",24,"""Nový Dvůr""",4,"""Nový Dvůr""",67,15.110943,50.232204,0.059701,0.358209,9
"""Čilá""",5,"""Čilá""",1,"""Čilá""",18,13.742122,49.961752,0.055556,0.277778,4
"""Louka""",51,"""Louka""",3,"""Louka""",61,16.431079,49.5292,0.04918,0.836066,5
"""Komárov""",72,"""Komárov""",13,"""Komárov""",309,17.568761,49.150692,0.042071,0.23301,7
"""Benátky""",86,"""Benátky""",5,"""Benátky""",122,15.736071,50.310237,0.040984,0.704918,7
"""Horní Vilémovice""",30,"""Horní Vilémovice""",3,"""Horní Vilémovice""",78,15.880829,49.291604,0.038462,0.384615,16
"""Budyně""",15,"""Budyně""",2,"""Budyně""",53,14.07118,49.146383,0.037736,0.283019,6


In [86]:
final.filter(pl.col("obyvatel") < 100000).sort(by="rodaku/obyvatel",descending=False).head(10)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Milovice""",18,"""Milovice""",1,"""Milovice""",13920,14.88871,50.226025,0.000072,0.001293,8
"""Krupka""",1,"""Krupka""",1,"""Krupka""",12779,13.858257,50.684577,0.000078,0.000078,6
"""Chodov""",13,"""Chodov""",1,"""Chodov""",12649,12.753404,50.241451,0.000079,0.001028,6
"""Jirkov""",8,"""Jirkov""",3,"""Jirkov""",19323,13.447786,50.499891,0.000155,0.000414,6
"""Chrastava""",6,"""Chrastava""",1,"""Chrastava""",6340,14.968948,50.816986,0.000158,0.000946,9
"""Františkovy Lázně""",1,"""Františkovy Lázně""",1,"""Františkovy Lázně""",5783,12.351815,50.120455,0.000173,0.000173,17
"""Holýšov""",10,"""Holýšov""",1,"""Holýšov""",5726,13.101339,49.59372,0.000175,0.001746,7
"""Uničov""",11,"""Uničov""",2,"""Uničov""",11151,17.121482,49.770929,0.000179,0.000986,6
"""Ludgeřovice""",2,"""Ludgeřovice""",1,"""Ludgeřovice""",4980,18.240131,49.890409,0.000201,0.000402,11


In [87]:
final.select(pl.col("rodaku/obyvatel")).median()

rodaku/obyvatel
f64
0.003252


In [88]:
final.filter(pl.col("rodna_obec") == "Hlinsko")

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Hlinsko""",304,"""Hlinsko""",48,"""Hlinsko""",9577,15.907606,49.762173,0.005012,0.031743,7


### Jednotlivé pohledy

In [90]:
def mapka(filtr: pl.DataFrame, vpravo=[], shift=[], pismo=9, pozadi='#FFFFE4', nadpis="", podnadpis="", bez_porodnic=False, kolik=10, posunY=7.5, posunX=0):

    if bez_porodnic == True:
        filtr = filtr.filter(~pl.col('rodna_obec').is_in(porodnice))

    filtr = filtr.head(kolik)
    
    print(filtr.select(pl.col("rodna_obec")).to_series().to_list())

    if podnadpis != None:
        titulkova_cast = alt.Title(nadpis, subtitle=podnadpis)
    else:
        titulkova_cast = alt.Title(nadpis)
    
    base_map = alt.Chart(
        alt.Data(values=json.loads(czech_geojson_json)['features']), width=kredity['sirka'], height=kredity['sirka'] / 1.5,
        title=titulkova_cast
    ).mark_geoshape(
        filled=True,
        fill=pozadi, #''#FFFFE4',  # Choose your desired fill color
        stroke='#EED801', #'#DB842F',
        strokeWidth=0.6
    )
    
    points_not_in = alt.Chart(
        filtr.filter(~pl.col('rodna_obec').is_in(porodnice)).to_pandas()
    ).mark_circle(
        color='#D6534B',  # red color
        size=25,
        opacity=1
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q')

    if bez_porodnic == False:
        points_in = alt.Chart(
            filtr.filter(pl.col('rodna_obec').is_in(porodnice)).to_pandas()
        ).mark_circle(
            color='#D6534B',
            opacity=0.5,
            size=25
        ).encode(
            longitude='longitude:Q',
            latitude='latitude:Q')
        
    labels_l = alt.Chart(
        filtr.filter(~pl.col('rodna_obec').is_in(vpravo)).filter(~pl.col('rodna_obec').is_in(shift)).to_pandas()
    ).mark_text(
        align='right',
        baseline='middle',
        dx=-5,
        dy=0,
        fontSize=pismo,
        font='Asap'
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        text='rodna_obec:N'
    )
    
    labels_r = alt.Chart(
        filtr.filter(pl.col('rodna_obec').is_in(vpravo)).filter(~pl.col('rodna_obec').is_in(shift)).to_pandas()
    ).mark_text(
        align='left',
        baseline='middle',
        dx=5,
        dy=0,
        fontSize=pismo,
        font='Asap'
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        text='rodna_obec:N'
    )

    labels_shift = alt.Chart(
        filtr.filter(pl.col('rodna_obec').is_in(shift)).to_pandas()
    ).mark_text(
        align='center',
        baseline='middle',
        dx=posunX,
        dy=posunY,
        fontSize=pismo,
        font='Asap'
    ).encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        text='rodna_obec:N'
    )

    if bez_porodnic == False:
        czech_map = (base_map + points_not_in + points_in + labels_r + labels_l + labels_shift)
    else:
        czech_map = (base_map + points_not_in + labels_r + labels_l + labels_shift)
    czech_map = czech_map.properties(
    width=300,
    height=200
)

    return czech_map


In [91]:
slepa = alt.Chart(
        alt.Data(values=json.loads(czech_geojson_json)['features']), width=300, height=200,
        title="slepá mapa"
    ).mark_geoshape(
        filled=True,
        fill='#FFFFE4', #''#FFFFE4',  # Choose your desired fill color
        stroke='#EED801', #'#DB842F',
        strokeWidth=0.6
    ).properties(
    width=300,
    height=200)

slepa

alt.Chart(...)

In [92]:
# me_to_neurazi(slepa, kredity="ahaha", soubor="slepa_mapa")

In [93]:
nad1000 = mapka(
    filtr=final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='knih/obyvatel',descending=True), 
    vpravo=["Klenčí pod Čerchovem","Malé Svatoňovice", "Kvasiny", "Litomyšl", "Přílepy", "Žebrák", "Hrabyně", "Studená"],
    nadpis=["Obce nad 1000 obyvatel podle","počtu spisovatelů na obyvatele"],
    kolik=15,
    bez_porodnic=True
)

nad1000

['Kněževes', 'Malé Svatoňovice', 'Nivnice', 'Klenčí pod Čerchovem', 'Přílepy', 'Nové Hrady', 'Kvasiny', 'Trhová Kamenice', 'Vysoké nad Jizerou', 'Sobotka', 'Hrabyně', 'Černčice', 'Studená', 'Počátky', 'Rataje']


alt.LayerChart(...)

In [94]:
final.columns

['rodna_obec',
 'knih',
 'rodna_obec_right',
 'rodaku',
 'name',
 'obyvatel',
 'longitude',
 'latitude',
 'rodaku/obyvatel',
 'knih/obyvatel',
 'delka_nazvu']

In [151]:
final.filter(pl.col("rodna_obec") == "Nivnice").filter(pl.col('obyvatel') > 3000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Nivnice""",457,"""Nivnice""",6,"""Nivnice""",3373,17.647573,48.974649,0.001779,0.135488,7


In [95]:
nad3000 = mapka(
    filtr=final.filter(pl.col('obyvatel') > 3000).drop_nulls().sort(by='rodaku/obyvatel',descending=True), 
    vpravo=["Hronov","Jaroměř", "Polička", "Litomyšl",'Vysoké Mýto','Přibyslav',"Polná","Humpolec",'Chlumec nad Cidlinou'],
    nadpis=["Obce s největším podílem rodáků-spisovatelů"],
    podnadpis=["Patnáct obcí nad 3000 obyvatel, kde není porodnice.",
               "Tím vypadlo Opočno, Nové Město na Moravě nebo Litomyšl,",
               "zároveň tento filtr posiluje zastoupení autorů 19. století."],
    bez_porodnic=True,
    kolik=15,
)

nad3000

['Valtice', 'Polička', 'Chlumec nad Cidlinou', 'Vysoké Mýto', 'Hořice', 'Dačice', 'Kutná Hora', 'Jaroměř', 'Humpolec', 'Pacov', 'Český Brod', 'Nový Bydžov', 'Lomnice nad Popelkou', 'Sušice', 'Skuteč']


alt.LayerChart(...)

In [147]:
df.filter(pl.col("rodna_obec") == "Nivnice").group_by("100_a").len()

100_a,len
str,u32
"""Prager, Ferdinand,""",3
"""Hladiš, František,""",2
"""Kachník, Josef,""",12
"""Komenský, Jan Amos,""",436
"""Ondra, Vlastimil,""",1
"""Zámečník, Stanislav,""",3


In [96]:
plodne_obce = ['Valtice', 'Polička', 'Dačice', 'Chlumec nad Cidlinou', 'Vysoké Mýto', 'Kutná Hora', 'Hořice', 'Jaroměř', 'Pacov', 'Humpolec', 'Skuteč', 'Český Brod', 'Nový Bydžov', 'Lomnice nad Popelkou', 'Horažďovice']
for p in plodne_obce:
    print(p.upper())
    print(df.filter(pl.col('rodna_obec') == p).group_by(['100_7','100_a']).len().sort(by='len',descending=True).head(3).select(pl.col('100_a')).to_series().to_list())

VALTICE
['Hlavenka, Jiří,', 'Trávníček, František,', 'Adámek, Zdeněk,']
POLIČKA
['Andrlík, František Josef,', 'Bureš, Miloslav,', 'Soukal, Josef,']
DAČICE
['Lanczová, Lenka,', 'Štěpán, Ludvík,', 'Holá, Lída,']
CHLUMEC NAD CIDLINOU
['Klicpera, Václav Kliment,', 'Richter, Karel,', 'Matějček, Zdeněk,']
VYSOKÉ MÝTO
['Martinková, Věra,', 'Kalandra, Čeněk,', 'Klíma, Jan,']
KUTNÁ HORA
['Tyl, Josef Kajetán,', 'Preissová, Gabriela,', 'Florian, Miroslav,']
HOŘICE
['Francková, Zuzana,', 'Lužická, Věnceslava,', 'Kuděj, Zdeněk Matěj,']
JAROMĚŘ
['Ivanov, Miroslav,', 'Bečvářová, Věra,', 'Dubský, Josef,']
PACOV
['Sova, Antonín,', 'Hájek, Jiří,', 'Pírko, Zdeněk,']
HUMPOLEC
['Kopáč, Jaroslav,', 'Suchánek, František,', 'Jirous, Ivan Martin,']
SKUTEČ
['Sahula, Jiří,', 'Tippmannová, Marie,', 'Petrus, Jan,']
ČESKÝ BROD
['Vokoun, Jaroslav Ignatius,', 'Šťastný, Karel,', 'Koman, Milan,']
NOVÝ BYDŽOV
['Jenčková, Eva,', 'Hlušička, Jiří,', 'Mádl, Karel Boromejský,']
LOMNICE NAD POPELKOU
['Bělina, Jan,', 'Zahradní

In [97]:
final.filter(pl.col('obyvatel') > 3000).drop_nulls().sort(by='rodaku/obyvatel',descending=True).head(10)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Nové Město n. M.""",621,"""Nové Město na Moravě""",176,"""Nové Město na Moravě""",9904,16.074221,49.561482,0.017771,0.062702,20
"""Jilemnice""",343,"""Jilemnice""",95,"""Jilemnice""",5443,15.506628,50.608945,0.017454,0.063017,9
"""Valtice""",191,"""Valtice""",59,"""Valtice""",3593,16.754981,48.740717,0.016421,0.053159,7
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Rychnov nad Kněžnou""",651,"""Rychnov nad Kněžnou""",154,"""Rychnov nad Kněžnou""",11442,16.274955,50.162871,0.013459,0.056896,19
"""Jičín""",1114,"""Jičín""",215,"""Jičín""",16230,15.351653,50.437045,0.013247,0.068638,5
"""Kyjov""",590,"""Kyjov""",141,"""Kyjov""",10799,17.122532,49.010193,0.013057,0.054635,5


In [98]:
final.filter(pl.col('obyvatel') > 3000).filter(~pl.col("rodna_obec").is_in(porodnice)).drop_nulls().sort(by='rodaku/obyvatel',descending=True).head(10)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Valtice""",191,"""Valtice""",59,"""Valtice""",3593,16.754981,48.740717,0.016421,0.053159,7
"""Polička""",584,"""Polička""",116,"""Polička""",8939,16.265479,49.714679,0.012977,0.065332,7
"""Chlumec nad Cidlinou""",375,"""Chlumec nad Cidlinou""",64,"""Chlumec nad Cidlinou""",5592,15.460335,50.154454,0.011445,0.06706,20
"""Vysoké Mýto""",737,"""Vysoké Mýto""",141,"""Vysoké Mýto""",12412,16.161752,49.95324,0.01136,0.059378,11
"""Hořice""",651,"""Hořice""",91,"""Hořice""",8587,15.631918,50.366137,0.010597,0.075812,6
"""Dačice""",330,"""Dačice""",76,"""Dačice""",7228,15.437279,49.081599,0.010515,0.045656,6
"""Kutná Hora""",1241,"""Kutná Hora""",215,"""Kutná Hora""",21556,15.268226,49.948445,0.009974,0.057571,10
"""Jaroměř""",699,"""Jaroměř""",106,"""Jaroměř""",12541,15.921447,50.356235,0.008452,0.055737,7
"""Humpolec""",538,"""Humpolec""",92,"""Humpolec""",11447,15.359357,49.541578,0.008037,0.046999,8


In [99]:
final.filter(pl.col('obyvatel') > 3000).filter(~pl.col("rodna_obec").is_in(porodnice)).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Nivnice""",457,"""Nivnice""",6,"""Nivnice""",3373,17.647573,48.974649,0.001779,0.135488,7
"""Hořice""",651,"""Hořice""",91,"""Hořice""",8587,15.631918,50.366137,0.010597,0.075812,6
"""Chlumec nad Cidlinou""",375,"""Chlumec nad Cidlinou""",64,"""Chlumec nad Cidlinou""",5592,15.460335,50.154454,0.011445,0.06706,20
"""Polička""",584,"""Polička""",116,"""Polička""",8939,16.265479,49.714679,0.012977,0.065332,7
"""Pacov""",296,"""Pacov""",38,"""Pacov""",4749,15.001714,49.470827,0.008002,0.062329,5
"""Vysoké Mýto""",737,"""Vysoké Mýto""",141,"""Vysoké Mýto""",12412,16.161752,49.95324,0.01136,0.059378,11
"""Kutná Hora""",1241,"""Kutná Hora""",215,"""Kutná Hora""",21556,15.268226,49.948445,0.009974,0.057571,10
"""Skuteč""",290,"""Skuteč""",36,"""Skuteč""",5042,15.996603,49.843508,0.00714,0.057517,6
"""Jaroměř""",699,"""Jaroměř""",106,"""Jaroměř""",12541,15.921447,50.356235,0.008452,0.055737,7


In [100]:
nad3000_chvost = mapka(
    filtr=final.filter(pl.col('obyvatel') > 3000).filter(pl.col("rodna_obec") != "Chodov").drop_nulls().sort(by='knih/obyvatel',descending=False), 
    vpravo=[],
    nadpis=["Obce s nejnižším podílem rodáků-spisovatelů"],
    podnadpis=None,
    bez_porodnic=False,
    kolik=15,
    shift=['Uničov','Fulnek']
)

nad3000_chvost

['Krupka', 'Františkovy Lázně', 'Nový Malín', 'Jiříkov', 'Rapotín', 'Lutín', 'Červená Voda', 'Nýrsko', 'Ludgeřovice', 'Jirkov', 'Vrbno pod Pradědem', 'Fulnek', 'Těrlicko', 'Mohelnice', 'Hrádek nad Nisou']


alt.LayerChart(...)

In [101]:
top_bez

rodna_obec,100_7,name,population,podil,population_right,longitude,latitude
str,u32,str,i64,f64,i64,f64,f64
null,null,"""Štětí""",8609,null,8609,14.374305,50.45306
null,null,"""Modřice""",5656,null,5656,16.614424,49.127917
null,null,"""Horní Slavkov""",5438,null,5438,12.807658,50.138745
null,null,"""Bor""",5132,null,5132,12.775216,49.71171
null,null,"""Meziboří""",4587,null,4587,13.598802,50.621246
null,null,"""Bolatice""",4501,null,4501,18.083636,49.951712
null,null,"""Kynšperk nad Ohří""",4496,null,4496,12.530346,50.119046
null,null,"""Chlumec""",4280,null,4280,13.939746,50.699804
null,null,"""Nová Role""",4244,null,4244,12.784302,50.27103


In [102]:
final.filter(pl.col("rodna_obec") == "Wegstädtl")

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64


In [103]:
help(mapka)

Help on function mapka in module __main__:

mapka(filtr: polars.dataframe.frame.DataFrame, vpravo=[], shift=[], pismo=9, pozadi='#FFFFE4', nadpis='', podnadpis='', bez_porodnic=False, kolik=10, posunY=7.5, posunX=0)



In [104]:
chvost = mapka(
    filtr=top_bez.drop('rodna_obec').rename({'name':'rodna_obec'}), 
    vpravo=['Horní Slavkov','Štětí','Chlumec','Modřice'],
    nadpis=["Největší obce bez českých spisovatelů mezi rodáky"],
    shift=['Kynšperk nad Ohří'],
    podnadpis=["Největší je s více než 8 tisíci obyvateli Štětí. Několik spisovatelů","v něm však žilo, například Bedřich Bernau nebo Josef Pecinovský."],
    bez_porodnic=False,
    kolik=8,
    posunY=-10.5,
    posunX=21
)

chvost

['Štětí', 'Modřice', 'Horní Slavkov', 'Bor', 'Meziboří', 'Bolatice', 'Kynšperk nad Ohří', 'Chlumec']


alt.LayerChart(...)

In [105]:
lokality

,name,wikidata_id,coordinates,population,area,region,district
1097,Praha,Q1085,Point(14.421388888 50.0875),1384732,496.210000,Czech Republic,Unknown
1143,Brno,Q14960,Point(16.608333333 49.195277777),400566,230.182739,Brno-City District,Unknown
1136,Ostrava,Q8385,Point(18.2925 49.835555555),284765,214.229602,Ostrava-City District,Unknown
1086,Plzeň,Q43453,Point(13.3825 49.741388888),185599,137.671042,Plzeň-City District,Unknown
952,Liberec,Q146351,Point(15.05 50.766666666),107982,106.087154,Liberec District,Unknown
...,...,...,...,...,...,...,...
526,Hill fort Chlum,Q110290703,Point(14.926382303 50.378408905),0,0.000000,Nepřevázka,Unknown
273,Q37453718,Q37453718,Point(13.8794367 49.0689152),0,0.000000,Lštění,Unknown
527,Q47035482,Q47035482,Point(13.51383 49.7611342),0,0.000000,Dýšina,Unknown
528,Q106630428,Q106630428,Point(14.5871 50.0557),0,0.000000,Dubeč,Unknown


In [143]:
final.filter(pl.col('obyvatel') > 3000).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Nivnice""",457,"""Nivnice""",6,"""Nivnice""",3373,17.647573,48.974649,0.001779,0.135488,7
"""Opočno""",343,"""Opočno""",57,"""Opočno""",3140,16.114874,50.267452,0.018153,0.109236,6
"""Litomyšl""",951,"""Litomyšl""",163,"""Litomyšl""",10493,16.310576,49.872064,0.015534,0.090632,8
"""Hořice""",651,"""Hořice""",91,"""Hořice""",8587,15.631918,50.366137,0.010597,0.075812,6
"""Čáslav""",783,"""Čáslav""",142,"""Čáslav""",10512,15.389777,49.911045,0.013508,0.074486,6
"""Jičín""",1114,"""Jičín""",215,"""Jičín""",16230,15.351653,50.437045,0.013247,0.068638,5
"""Chlumec nad Cidlinou""",375,"""Chlumec nad Cidlinou""",64,"""Chlumec nad Cidlinou""",5592,15.460335,50.154454,0.011445,0.06706,20
"""Polička""",584,"""Polička""",116,"""Polička""",8939,16.265479,49.714679,0.012977,0.065332,7
"""Boskovice""",777,"""Boskovice""",143,"""Boskovice""",12190,16.660002,49.487537,0.011731,0.063741,9


In [106]:
nad3000_knihy = mapka(
    filtr=final.filter(pl.col('obyvatel') > 3000).drop_nulls().sort(by='knih/obyvatel',descending=True), 
    vpravo=['Jaroměř','Polná','Přibyslav','Vysoké Mýto','Polička'],
    nadpis=["Obce s největším počtem publikací na obyvatele"],
    podnadpis=["Opět sídla s více než 3000 obyvateli, opět bez porodnic."],
    bez_porodnic=True,
    kolik=15
)

nad3000_knihy

['Nivnice', 'Hořice', 'Chlumec nad Cidlinou', 'Polička', 'Pacov', 'Vysoké Mýto', 'Kutná Hora', 'Skuteč', 'Jaroměř', 'Valtice', 'Přibyslav', 'Humpolec', 'Polná', 'Dačice', 'Hronov']


alt.LayerChart(...)

In [107]:
hustle_obce = ['Nivnice', 'Hořice', 'Chlumec nad Cidlinou', 'Polička', 'Pacov', 'Skuteč', 'Vysoké Mýto', 'Kutná Hora', 'Jaroměř', 'Valtice', 'Přibyslav', 'Humpolec', 'Polná', 'Dačice', 'Hronov']

In [108]:
for h in hustle_obce:
    print(h.upper())
    print(df.filter(pl.col('rodna_obec') == h).group_by(['100_7','100_a']).len().sort(by='len',descending=True).head(5).select(pl.col('100_a')).to_series().to_list())

NIVNICE
['Komenský, Jan Amos,', 'Kachník, Josef,', 'Prager, Ferdinand,', 'Zámečník, Stanislav,', 'Hladiš, František,']
HOŘICE
['Francková, Zuzana,', 'Lužická, Věnceslava,', 'Kuděj, Zdeněk Matěj,', 'Baudyš, Eduard,', 'Máček, Miloš,']
CHLUMEC NAD CIDLINOU
['Klicpera, Václav Kliment,', 'Richter, Karel,', 'Matějček, Zdeněk,', 'Váňová, Magda,', 'Kuča, Karel,']
POLIČKA
['Andrlík, František Josef,', 'Bureš, Miloslav,', 'Soukal, Josef,', 'Dudek, Adolf,', 'Spalová, Olga,']
PACOV
['Sova, Antonín,', 'Hájek, Jiří,', 'Pírko, Zdeněk,', 'Vondráček, Jan,', 'Lapáček, Jiří,']
SKUTEČ
['Sahula, Jiří,', 'Tippmannová, Marie,', 'Petrus, Jan,', '<<z >>Finberka, Jan Václav,', 'Holubář, Josef,']
VYSOKÉ MÝTO
['Martinková, Věra,', 'Kalandra, Čeněk,', 'Klíma, Jan,', 'Brůna, Otakar,', 'Kolisko, Hugo Theodor,']
KUTNÁ HORA
['Tyl, Josef Kajetán,', 'Preissová, Gabriela,', 'Florian, Miroslav,', 'Virius, Miroslav,', 'Domin, Karel,']
JAROMĚŘ
['Ivanov, Miroslav,', 'Bečvářová, Věra,', 'Dubský, Josef,', 'Veselovský, Zdeněk,'

In [159]:
combined_chart = nad3000.configure_projection( # , alt.vconcat(hustota_do_grafu, nad3000, chvost)
    type='mercator',
    scale=3500,
    center=[15.5, 49.8]).properties(
    autosize={'contains': 'padding'}
)

me_to_neurazi(combined_chart, kredity=kredity['default'], soubor="02_mapa_rodactva")

combined_chart

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/02_mapa_rodactva.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/02_mapa_rodactva.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


alt.LayerChart(...)

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/02_mapa_rodactva.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/02_mapa_rodactva.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [111]:
final.sort(by="rodaku")

rodna_obec,knih,rodna_obec_right,rodaku,name,obyvatel,longitude,latitude,rodaku/obyvatel,knih/obyvatel,delka_nazvu
str,u32,str,u32,str,i64,f64,f64,f64,f64,i64
"""Michalovice""",1,"""Michalovice""",1,"""Michalovice""",221,15.524361,49.576986,0.004525,0.004525,11
"""Bezuchov""",1,"""Bezuchov""",1,"""Bezuchov""",187,17.6088,49.462731,0.005348,0.005348,8
"""Domašov nad Bystřicí""",2,"""Domašov nad Bystřicí""",1,"""Domašov nad Bystřicí""",491,17.44534,49.742305,0.002037,0.004073,20
"""Tištín""",1,"""Tištín""",1,"""Tištín""",465,17.165417,49.306902,0.002151,0.002151,6
"""Sehradice""",5,"""Sehradice""",1,"""Sehradice""",684,17.824167,49.144267,0.001462,0.00731,9
"""Jankovice""",8,"""Jankovice""",1,"""Jankovice""",417,17.621629,49.356148,0.002398,0.019185,9
"""Janoušov""",2,"""Janoušov""",1,"""Janoušov""",53,16.845164,50.013132,0.018868,0.037736,8
"""Řisuty""",2,"""Řisuty""",1,"""Řisuty""",412,14.005057,50.216572,0.002427,0.004854,6
"""Zběšičky""",1,"""Zběšičky""",1,"""Zběšičky""",163,14.426502,49.392578,0.006135,0.006135,8


In [112]:
df.filter(pl.col("rodna_obec") == "Malé Svatoňovice").group_by(["100_a","100_7"]).len()

100_a,100_7,len
str,str,u32
"""Kirschnerová, Milena,""","""jk01060207""",2
"""Vihan, Karel,""","""jk01142315""",8
"""Baudyš, Petr,""","""mzk2003173781""",3
"""Holub, Ota,""","""jk01041820""",42
"""Řehák, Karel,""","""mzk2018986488""",1
"""Čapek, Karel,""","""jk01021023""",153
"""Schejbal, Ctirad,""","""jn20000710497""",19


In [113]:
final.columns

['rodna_obec',
 'knih',
 'rodna_obec_right',
 'rodaku',
 'name',
 'obyvatel',
 'longitude',
 'latitude',
 'rodaku/obyvatel',
 'knih/obyvatel',
 'delka_nazvu']

In [114]:
mapka(
    filtr=final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='rodaku/obyvatel',descending=True).head(10), 
    vpravo=["Klenčí pod Čerchovem","Malé Svatoňovice"]
)

['Čeladná', 'Přílepy', 'Opočno', 'Nové Město n. M.', 'Jilemnice', 'Vysoké nad Jizerou', 'Valtice', 'Městec Králové', 'Kněževes', 'Počátky']


alt.LayerChart(...)

In [115]:
mapka(filtr=graf2, vpravo=['Jilemnice','Opočno'])

['Čeladná', 'Přílepy', 'Opočno', 'Nové Město n. M.', 'Jilemnice', 'Vysoké nad Jizerou', 'Valtice', 'Městec Králové', 'Kněževes', 'Počátky']


alt.LayerChart(...)

In [116]:
mapka(filtr=graf3)

['Litomyšl', 'Čáslav', 'Rychnov nad Kněžnou', 'Jičín', 'Kyjov', 'Boskovice', 'Vysoké Mýto', 'Turnov', 'Uherské Hradiště', 'Kutná Hora']


alt.LayerChart(...)

In [117]:
mapka(
    filtr=final.filter(pl.col('obyvatel') > 8000).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10),
    vpravo=['Hořice','Boskovice','Vysoké Mýto','Litomyšl','Polička','Čáslav']
)

['Litomyšl', 'Hořice', 'Čáslav', 'Jičín', 'Polička', 'Boskovice', 'Nové Město n. M.', 'Písek', 'Vysoké Mýto', 'Turnov']


alt.LayerChart(...)

In [118]:
mapka(
    filtr=final.filter(pl.col('obyvatel') > 8000).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10),
    vpravo=['Hořice','Boskovice','Vysoké Mýto','Litomyšl','Polička','Čáslav']
)

['Litomyšl', 'Hořice', 'Čáslav', 'Jičín', 'Polička', 'Boskovice', 'Nové Město n. M.', 'Písek', 'Vysoké Mýto', 'Turnov']


alt.LayerChart(...)

In [119]:
mapka(
    filtr=final.filter(pl.col('obyvatel') > 8000).filter(~pl.col("rodna_obec").is_in(porodnice)).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10),
    vpravo=['Hořice','Boskovice','Vysoké Mýto','Litomyšl','Polička','Čáslav']
)

['Hořice', 'Polička', 'Vysoké Mýto', 'Kutná Hora', 'Jaroměř', 'Humpolec', 'Uherský Brod', 'Louny', 'Nová Paka', 'Hlinsko']


alt.LayerChart(...)

In [120]:
mapka(
    filtr=final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='knih/obyvatel',descending=True).head(10),
    vpravo=['Hořice','Boskovice','Vysoké Mýto','Litomyšl','Polička','Čáslav']
)

['Kněževes', 'Malé Svatoňovice', 'Nivnice', 'Klenčí pod Čerchovem', 'Přílepy', 'Opočno', 'Nové Hrady', 'Kvasiny', 'Trhová Kamenice', 'Vysoké nad Jizerou']


alt.LayerChart(...)